Taken from [this tutorial](https://github.com/keras-team/keras-io/blob/master/examples/vision/knowledge_distillation.py)

In [1]:
"""
Title: Knowledge Distillation
Author: [Kenneth Borup](https://twitter.com/Kennethborup)
Date created: 2020/09/01
Last modified: 2020/09/01
Description: Implementation of classical Knowledge Distillation.
"""

"""
## Introduction to Knowledge Distillation
Knowledge Distillation is a procedure for model
compression, in which a small (student) model is trained to match a large pre-trained
(teacher) model. Knowledge is transferred from the teacher model to the student
by minimizing a loss function, aimed at matching softened teacher logits as well as
ground-truth labels.
The logits are softened by applying a "temperature" scaling function in the softmax,
effectively smoothing out the probability distribution and revealing
inter-class relationships learned by the teacher.
**Reference:**
- [Hinton et al. (2015)](https://arxiv.org/abs/1503.02531)
"""

"""
## Setup
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
"""
## Construct `Distiller()` class
The custom `Distiller()` class, overrides the `Model` methods `train_step`, `test_step`,
and `compile()`. In order to use the distiller, we need:
- A trained teacher model
- A student model to train
- A student loss function on the difference between student predictions and ground-truth
- A distillation loss function, along with a `temperature`, on the difference between the
soft student predictions and the soft teacher labels
- An `alpha` factor to weight the student and distillation loss
- An optimizer for the student and (optional) metrics to evaluate performance
In the `train_step` method, we perform a forward pass of both the teacher and student,
calculate the loss with weighting of the `student_loss` and `distillation_loss` by `alpha` and
`1 - alpha`, respectively, and perform the backward pass. Note: only the student weights are updated,
and therefore we only calculate the gradients for the student weights.
In the `test_step` method, we evaluate the student model on the provided dataset.
"""


class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.
        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [4]:
"""
## Prepare the dataset
The dataset used for training the teacher and distilling the teacher is
[MNIST](https://keras.io/api/datasets/mnist/), and the procedure would be equivalent for any other
dataset, e.g. [CIFAR-10](https://keras.io/api/datasets/cifar10/), with a suitable choice
of models. Both the student and teacher are trained on the training set and evaluated on
the test set.
"""

# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [5]:
from skimage.transform import resize

def resize_img(data):
    output = np.zeros((data.shape[0],64))
    for i, img in enumerate(data):
        output[i]=resize(img, (8,8)).flatten()
    return output

x_train_small = resize_img(x_train)
y_train_small = resize_img(y_train)
x_test_small = resize_img(x_test)
y_test_small = resize_img(y_test)


#bottle_resized = resize(bottle, (140, 54))

In [6]:
# Normalize data
x_train = x_train_small.astype("float32") / 255.0
x_train = np.reshape(x_train_small, (-1, 64))

x_test = x_test_small.astype("float32") / 255.0
x_test = np.reshape(x_test_small, (-1, 64))

In [54]:
"""
## Create student and teacher models
Initialy, we create a teacher model and a smaller student model. Both models are
convolutional neural networks and created using `Sequential()`,
but could be any Keras model.
"""

# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(64,)),
        layers.Dense(100, activation='relu'),
        layers.Dense(80, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(40, activation='relu'),
        layers.Dense(20, activation='relu'),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(64,)),
        layers.Dense(40, activation='relu'),
        layers.Dense(40, activation='relu'),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [55]:
"""
## Train the teacher
In knowledge distillation we assume that the teacher is trained and fixed. Thus, we start
by training the teacher model on the training set in the usual way.
"""

# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4302 - sparse_categorical_accuracy: 0.8623
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1848 - sparse_categorical_accuracy: 0.9427
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1471 - sparse_categorical_accuracy: 0.9536
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1261 - sparse_categorical_accuracy: 0.9609
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1120 - sparse_categorical_accuracy: 0.9639
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1014 - sparse_categorical_accuracy: 0.9674
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0915 - sparse_categorical_accuracy: 0.9700
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0857 - sparse_categorical_accuracy: 0.9723
Epoch 9/10
1875/1875 [==========

In [56]:
teacher.evaluate(x_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.0999 - sparse_categorical_accuracy: 0.9706


[0.0999349057674408, 0.9706000089645386]

In [57]:
teacher.save("mnist8x8_100_80_60_40_20_10.h5")

In [58]:
"""
## Distill teacher to student
We have already trained the teacher model, and we only need to initialize a
`Distiller(student, teacher)` instance, `compile()` it with the desired losses,
hyperparameters and optimizer, and distill the teacher to the student.
"""

# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=30)

Epoch 1/30
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.8329 - student_loss: 0.5669 - distillation_loss: 0.0675
Epoch 2/30
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9074 - student_loss: 0.3001 - distillation_loss: 0.0323
Epoch 3/30
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9276 - student_loss: 0.2331 - distillation_loss: 0.0229
Epoch 4/30
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9389 - student_loss: 0.1936 - distillation_loss: 0.0176
Epoch 5/30
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9463 - student_loss: 0.1710 - distillation_loss: 0.0147
Epoch 6/30
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9517 - student_loss: 0.1554 - distillation_loss: 0.0129
Epoch 7/30
1875/1875 [==============================] - 4s

In [59]:
# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

313/313 [==============================] - 0s 1ms/step - sparse_categorical_accuracy: 0.9691 - student_loss: 0.1025


0.9690999984741211

In [60]:
student.save("mnist8x8_100_80_60_40_20_10-mirror.h5")

In [61]:
from tensorflow.python.keras.models import load_model
import onnx
import keras2onnx

In [62]:
model = load_model('mnist8x8_100_80_60_40_20_10.h5')
onnx_model = keras2onnx.convert_keras(model, model.name)
onnx.save_model(onnx_model, "mnist8x8_100_80_60_40_20_10.onnx")

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 22 -> 17
The maximum opset needed by this model is only 9.


In [63]:
model = load_model('mnist8x8_100_80_60_40_20_10-mirror.h5')
onnx_model = keras2onnx.convert_keras(model, model.name)
onnx.save_model(onnx_model, "mnist8x8_100_80_60_40_20_10-mirror.onnx")

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 13 -> 8
The maximum opset needed by this model is only 9.


Note: The student network was trained for a total of 18 epochs!


In [71]:
#point = [0.0, 0.01, 8.005, 14.005, 14.995, 13.995, 7.005, 0.01, 0.0, 0.0, 6.995, 7.995, 8.995, 14.995, 8.005, 0.0, 0.01, 0.0, 0.0, 0.01, 5.005, 13.995, 2.995, 0.0, 0.0, 0.0, 5.005, 9.005, 13.995, 13.005, 4.005, 0.0, 0.0, 1.005, 10.005, 14.005, 14.995, 12.005, 4.005, 0.01, 0.0, 0.0, 2.005, 13.005, 8.005, 1.995, 0.0, 0.01, 0.0, 0.0, 5.995, 14.995, 2.005, 0.01, 0.0, 0.01, 0.01, 0.0, 9.995, 9.995, 0.01, 0.01, 0.01, 0.01]
#point = [0.0, 0.01, 8.005, 14.005, 14.995, 13.995, 7.005, 0.01, 0.0, 0.0, 6.995, 7.995, 8.995, 14.995, 8.005, 0.0, 0.01, 0.0, 0.0, 0.01, 5.005, 13.995, 2.995, 0.0, 0.0, 0.0, 5.005, 9.005, 13.995, 13.005, 4.005, 0.0, 0.0, 1.005, 10.005, 14.005, 14.995, 12.005, 4.005, 0.01, 0.0, 0.0, 2.005, 13.005, 8.005, 1.995, 0.0, 0.01, 0.0, 0.0, 5.995, 14.995, 2.005, 0.01, 0.0, 0.01, 0.01, 0.0, 9.995, 9.995, 0.01, 0.01, 0.01, 0.01]
#point = [0.0, 0.01, 8.005, 14.005, 14.995, 13.995, 7.005, 0.01, 0.0, 0.0, 6.995, 7.995, 8.995, 14.995, 8.005, 0.0, 0.01, 0.0, 0.0, 0.01, 5.005, 13.995, 2.995, 0.0, 0.0, 0.0, 5.005, 9.005, 13.995, 13.005, 4.005, 0.0, 0.0, 1.005, 10.005, 14.005, 14.995, 12.005, 4.005, 0.01, 0.0, 0.0, 2.005, 13.005, 8.005, 1.995, 0.0, 0.01, 0.0, 0.0, 5.995, 14.995, 2.005, 0.01, 0.0, 0.01, 0.01, 0.0, 9.995, 9.995, 0.01, 0.01, 0.01, 0.01]
point = [0.1, 0.1, 7.95, 14.05, 14.95, 14.05, 6.95, 0.1, 0.0, 0.1, 6.95, 8.05, 9.05, 14.95, 8.05, 0.1, 0.1, 0.0, 0.1, 0.1, 5.05, 14.05, 2.95, 0.0, 0.1, 0.0, 4.95, 9.05, 14.05, 12.95, 4.05, 0.0, 0.0, 1.05, 9.95, 14.05, 14.95, 12.05, 4.05, 0.1, 0.1, 0.1, 2.05, 13.05, 8.05, 2.05, 0.0, 0.1, 0.0, 0.0, 6.05, 15.05, 2.05, 0.0, 0.1, 0.1, 0.1, 0.1, 9.95, 10.05, 0.1, 0.1, 0.1, 0.0]
teacher1 = load_model('mnist8x8_100_80_60_40_20_10.h5')
student1 = load_model('mnist8x8_100_80_60_40_20_10-mirror.h5')
p1=teacher1.predict(np.array([point]))
p2=student1.predict(np.array([point]))
print(np.argmax(p1)==np.argmax(p2))
print(p1)
print(p2)


True
[[-440.28275   216.44778   248.75108   407.42728  -185.77899    14.428975
  -505.653     117.48717    23.33815  -311.39465 ]]
[[-146.83609  -150.80287    94.99972   243.09435  -368.1465   -213.94551
  -328.0137    -70.71278   -53.830566 -466.51904 ]]


# Equivalence Properties
It seems that the `9200.e` properties with `e<5` are top 1 equivalent.
`9000*` and `9100*` on the other hand are not top 1 equivalent